#### **Import modules**

In [2]:
import re
from typing import List

from gensim.models import Word2Vec
from nltk import tokenize

from utils import GeorgianWord2Vec

#### **Init word2vec model**

In [39]:
texts = [["გამარჯობა", "როგორ", "ხარ?"], ["რავი", "კარგად", "შენ?"]]

model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)

model.save("word2vec.model")

## **Convert file into input format**

word2vec model gets list of list of strings as an input

In [40]:
def convert_file_into_input(file_path: str) -> List[List[str]]:
    sentences_split_regex = "\. |!|\?"
    with open(file_path, 'r') as f:
        data = f.read()
    sentences = re.split(sentences_split_regex, data)

    fn = lambda s: len(s) > 0
    return list(
        filter(fn, [list(filter(fn, [word.strip() for word in sentence.split(" ")])) for sentence in sentences]))

In [35]:
print(convert_file_into_input("test.txt"))

[['ბუნებრივი', 'ენის', 'ტექსტების', 'კომპიუტერული', 'დამუშავება', '—', 'თანამედროვე', 'საინფორმაციო', 'ტექნოლოგიების', 'ერთ-ერთი', 'მიმართულება.\nიგი', 'გულისხმობს:', 'მართლწერის', 'ავტომატურ', 'შემოწმებას;', 'მანქანურ', 'თარგმანს;', 'საძიებო', 'სისტემებს,', 'რომელთა', 'მეშვეობითაც', 'შესაძლებელია\nსიტყვის', 'სხვადასხვა', 'ფორმების', 'მოძებნა', 'ბაზაში;', 'ბუნებრივ', 'ენაზე', 'დიალოგურ', 'რეჟიმში', 'მომუშავე', 'პროგრამებს;', 'ტექსტების\nმრავალმიზნობრივ', 'მორფოლოგიურ,', 'სინტაქსურ', 'და', 'სემანტიკურ', 'ანალიზს;', 'ენის', 'სწავლებისათვის', 'განკუთვნილ', 'სავარჯიშო', 'ტრენაჟორებს', 'და\nსხვას.']]


In [2]:
print(model.wv["ავთო"])

NameError: name 'model' is not defined

In [3]:
georgian_model = GeorgianWord2Vec()

Initializing data


In [4]:
georgian_model.train("../data/data2.txt")

['ედუარდ', 'შევარდნაძე', 'ედუარდ', 'ამბროსის', 'ძე', 'შევარდნაძე', '(დ']


In [5]:
print(georgian_model.get_vector("ედუარდ"))

[ 8.1248647e-03 -4.4547413e-03 -1.0500057e-03  1.0474137e-03
 -1.9329190e-04  1.1259809e-03  6.1145429e-03 -3.5712287e-07
 -3.2579873e-03 -1.5135300e-03  5.8782487e-03  1.5009199e-03
 -7.3593750e-04  9.3621546e-03 -4.9140980e-03 -8.2872139e-04
  9.1967331e-03  6.7400974e-03  1.5266223e-03 -8.9173531e-03
  1.1553584e-03 -2.2661486e-03  9.3517248e-03  1.1999665e-03
  1.4676420e-03  2.4239894e-03 -1.8455875e-03 -5.0146668e-03
  2.4953051e-04 -2.0063855e-03  6.6344417e-03  8.9559210e-03
 -6.8635133e-04  2.9842837e-03 -6.1322823e-03  1.6900052e-03
 -6.9335690e-03 -8.6943740e-03 -5.9239641e-03 -8.9979367e-03
  7.2881677e-03 -5.7598399e-03  8.2939267e-03 -7.2441865e-03
  3.4343877e-03  9.7083794e-03 -7.8129116e-03 -9.9352738e-03
 -4.3090261e-03 -2.6836288e-03 -2.7622559e-04 -8.8360105e-03
 -8.6303856e-03  2.8191048e-03 -8.2509629e-03 -9.0519758e-03
 -2.3441375e-03 -8.6424854e-03 -7.0661777e-03 -8.4284320e-03
 -2.9962618e-04 -4.5501655e-03  6.5882672e-03  1.5387562e-03
 -3.3833650e-03  6.13094